<a href="https://colab.research.google.com/github/minurasam/Surface-Reconstruction_CV_pytorch/blob/main/CSC492592_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programming Assignment 3: Object Detection Model Experimentation

In this programming assignment, you are tasked with conducting experiments on object detection models, focusing on the differences between single-stage (**YOLO-family**) and two-stage (**RCNN-family**) approaches. Please note that the emphasis of this assignment is on **experimentation and analysis**, rather than on programming itself.

You will use the [COCO](https://cocodataset.org/#home) dataset as a public testbench to analyze their performance across various metrics. This assignment provides an opportunity to learn how to evaluate deep learning-based object detection models and understand how to appropriately select a model for potential future applications.

***Please run the prepared codes below to answer the following questions:***

- [**50 pts**] Using a `sample size` of **400**, compare the performance of each model in terms of the following metrics: `mAP`, `f1-score`, `confusion matrix`, and `execution time` (inference speed per second).
 using plots and provide your interpretation on the results with your own opinion.
- [**25 pts**] For each model, evaluate its performance across different sample sizes: [**50,100,200,400**], focusing on metrics such as `mAP`, `F1-score`, and the `confusion matrix`. Please provide your interpretation of the results, especially from the perspective of **class balance**. Discuss how variations in sample size might affect model performance for different classes and share your insights on this matter.
- [**25 pts**] Please write a short paragraph describing your opinion on usecase scenarios or applications where a **two-stage** model might be preferred over a **single-stage** model. Consider factors such as real-time processing requirements, accuracy needs, computational resources, and the specific characteristics of the task or dataset.

***Submission:***
- Ensure your report is well-organized, with clear sections for each task.
- Ensure that your analysis reflects your own interpretation, rather than merely comparing numerical values.
- Submit your report in PDF format to D2L by ***April 10, 2024***.


***Additional Resources:***
- [YoloV5: Tutorial by ultralytics](https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb#scrollTo=1NcFxRcFdJ_O)
- [Faster-RCNN: Tutorial by PyTorch](https://pytorch.org/vision/master/models/generated/torchvision.models.detection.fasterrcnn_resnet50_fpn.html)
- [The COCO Dataset: Best Practices for Downloading, Visualization, and Evaluation](https://medium.com/voxel51/the-coco-dataset-best-practices-for-downloading-visualization-and-evaluation-68a3d7e97fb7)

In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [ ]:
# Install package
!pip install fiftyone

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

# Load coco-2017 validation dataset
######################
# TODO: Sample items #
######################
dataset = foz.load_zoo_dataset("coco-2017",
                               split="validation",
                               label_types=["detections"],
                               shuffle=True,
                               max_samples=400,
                               )

# Load model from zoo and apply it to dataset
######################
# TODO: Load a model #
######################
model1 = foz.load_zoo_model("yolov5n-coco-torch") # Model size: 7.76 KB
model2 = foz.load_zoo_model("yolov8n-coco-torch") # Model size: 6.23 MB
model3 = foz.load_zoo_model("faster-rcnn-resnet50-fpn-coco-torch") # Model size: 207.71 MB
# model4 = foz.load_zoo_model("mask-rcnn-resnet50-fpn-coco-torch")   # Model size: 169.84 MB

In [ ]:
session = fo.launch_app(dataset)

In [ ]:
import sys
sys.path.append("/content/root/")

In [ ]:
!pip install pillow==7

## Sample Size 400 Model 1

In [ ]:
dataset.apply_model(model2, label_field="predictions")

# Evaluate `predictions` w.r.t. labels in `ground_truth` field
######################################################################
# TODO: Evaluate the model with IoU (Intersection over Union) of 0.5 #
######################################################################
results = dataset.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
    iou=0.5,
)


#########################################################
# TODO: Analyze COCO mAP (mean Average Precision) score #
#########################################################
mapscore = results.mAP()
print(f"mAP: {mapscore}")


# Print a classification report for the top-10 classes
results.print_report()

# Plot a confusion matrix
plot = results.plot_confusion_matrix()
plot.show()

In [ ]:
classes = ["person", "kite", "car", "bird", "carrot", "chair", "bowl", "bottle", "book", "cup", "dining table", "umbrella"]

# Print a classification report for the top-10 classes
results.print_report(classes=classes)

# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_tp"))
print("FP: %d" % dataset.sum("eval_fp"))
print("FN: %d" % dataset.sum("eval_fn"))

print("FN: %d" % dataset.sum("eval_fn"))

# Create a view that has samples with the most false positives first, and
# only includes false positive boxes in the `predictions` field
view = (
    dataset
    .sort_by("eval_fp", reverse=True)
    .filter_labels("predictions", F("eval") == "fp")
)

plot = results.plot_confusion_matrix(classes=classes)
plot.show()

# Plot precision-recall curve
plot2 = results.plot_pr_curves(classes=classes)
plot2.show()

In [ ]:
dataset.apply_model(model1, label_field="predictions")

# Evaluate `predictions` w.r.t. labels in `ground_truth` field
######################################################################
# TODO: Evaluate the model with IoU (Intersection over Union) of 0.5 #
######################################################################
results = dataset.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
    iou=0.5,
)


#########################################################
# TODO: Analyze COCO mAP (mean Average Precision) score #
#########################################################
mapscore = results.mAP()
print(f"mAP: {mapscore}")


# Print a classification report for the top-10 classes
results.print_report()

# Plot a confusion matrix
plot = results.plot_confusion_matrix()
plot.show()

In [ ]:
classes = ["person", "kite", "car", "bird", "carrot", "chair", "bowl", "bottle", "book", "cup", "dining table", "umbrella"]

# Print a classification report for the top-10 classes
results.print_report(classes=classes)

# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_tp"))
print("FP: %d" % dataset.sum("eval_fp"))
print("FN: %d" % dataset.sum("eval_fn"))

print("FN: %d" % dataset.sum("eval_fn"))

# Create a view that has samples with the most false positives first, and
# only includes false positive boxes in the `predictions` field
view = (
    dataset
    .sort_by("eval_fp", reverse=True)
    .filter_labels("predictions", F("eval") == "fp")
)

plot = results.plot_confusion_matrix(classes=classes)
plot.show()

# Plot precision-recall curve
plot2 = results.plot_pr_curves(classes=classes)
plot2.show()

In [ ]:
plot = results.plot_pr_curves(classes=["person", "kite", "car"])
plot.show()

In [ ]:
# Create a view that has samples with the most false positives first, and
# only includes false positive boxes in the `predictions` field
view = (
    dataset
    .sort_by("eval_fp", reverse=True)
    .filter_labels("predictions", F("eval") == "fp")
)

# Visualize results in the App
session = fo.launch_app(view=view)

## Faster RCNN

In [ ]:
dataset.apply_model(model3, label_field="predictions")

# Evaluate `predictions` w.r.t. labels in `ground_truth` field
######################################################################
# TODO: Evaluate the model with IoU (Intersection over Union) of 0.5 #
######################################################################
results = dataset.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
    iou=0.5,
)


#########################################################
# TODO: Analyze COCO mAP (mean Average Precision) score #
#########################################################
mapscore = results.mAP()
print(f"mAP: {mapscore}")


# Print a classification report for the top-10 classes
results.print_report()

# Plot a confusion matrix
plot = results.plot_confusion_matrix()
plot.show()

In [ ]:
classes = ["person", "kite", "car", "bird", "carrot", "chair", "bowl", "bottle", "book", "cup", "dining table", "umbrella"]

# Print a classification report for the top-10 classes
results.print_report(classes=classes)

# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_tp"))
print("FP: %d" % dataset.sum("eval_fp"))
print("FN: %d" % dataset.sum("eval_fn"))

print("FN: %d" % dataset.sum("eval_fn"))

# Create a view that has samples with the most false positives first, and
# only includes false positive boxes in the `predictions` field
view = (
    dataset
    .sort_by("eval_fp", reverse=True)
    .filter_labels("predictions", F("eval") == "fp")
)

plot = results.plot_confusion_matrix(classes=classes)
plot.show()

# Plot precision-recall curve
plot2 = results.plot_pr_curves(classes=classes)
plot2.show()